# Clustering Neighborhoods in Toronto

## Part 1: Wikipedia Scraping

Import Libraries

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

Parse the Wikipedia data

In [2]:
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(url,'lxml')
table = soup.find('table',{'class':'wikitable sortable'})
table_rows = table.findAll('tr')

Iterate through rows and append them to list

In [3]:
list_rows =[]
for tr in table_rows:
    td = tr.findAll('td')
    row = [tr.text for tr in td]
    list_rows.append(row)

Create Pandas dataframe
The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [4]:
df = pd.DataFrame(list_rows, columns=["Postcode","Borough","Neighborhood"])
df.replace('\n','',regex=True, inplace=True)

Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [5]:
df.dropna(axis=0, inplace=True)
not_assigned = df[df['Borough'] == 'Not assigned'].index
df.drop(not_assigned, inplace = True)

If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [6]:
df.loc[df.Neighborhood == 'Not assigned', 'Neighborhood'] = df.Borough

More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [7]:
df_cleansed = df.groupby(['Postcode','Borough'])['Neighborhood'].apply(lambda x: ', '.join(x.astype(str))).reset_index()

In [8]:
df_cleansed.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [9]:
df_cleansed.shape

(103, 3)

## Part 2: Geo-coding Postal Codes

Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

### 1. First Approach Using Geocoder

In [10]:
## This is the attempt using geocoder, unfortunately the results included too many NaNs
import geocoder
from tqdm import tqdm

lat = []
lng = []


for postcode in df_cleansed['Postcode']:
    g = geocoder.google(postcode,
                     components='city:Toronto',
                     key='API KEY')
    lat.append(g.lat)
    lng.append(g.lng)
    
df_cleansed['Latitude'] = lat
df_cleansed['Longitude'] = lng

df_cleansed[np.isnan(df_cleansed.Latitude)]

,Postcode,Borough,Neighborhood,Latitude,Longitude
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",NaN,NaN
9,M1N,Scarborough,"Birch Cliff, Cliffside West",NaN,NaN
13,M1T,Scarborough,"Clarks Corners, Sullivan, Tam O'Shanter",NaN,NaN
16,M1X,Scarborough,Upper Rouge,NaN,NaN
17,M2H,North York,Hillcrest Village,NaN,NaN
22,M2N,North York,Willowdale South,NaN,NaN
31,M3L,North York,Downsview West,NaN,NaN
33,M3N,North York,Downsview Northwest,NaN,NaN
34,M4A,North York,Victoria Village,NaN,NaN
35,M4B,East York,"Woodbine Gardens, Parkview Hill",NaN,NaN


### 2. Second Approach Using Cocl Data

In [10]:
## Read information from Cocl
cocl_url = 'http://cocl.us/Geospatial_data'
geocodes = pd.read_csv(cocl_url)
geocodes.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:
## Merge geocodes and df_cleansed
geocodes.rename(columns={'Postal Code' : 'Postcode'}, inplace=True)
df_geocodes = pd.merge(geocodes, df_cleansed, on = 'Postcode')
df_geocodes.head()

,Postcode,Latitude,Longitude,Borough,Neighborhood
0,M1B,43.806686,-79.194353,Scarborough,"Rouge, Malvern"
1,M1C,43.784535,-79.160497,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,43.770992,-79.216917,Scarborough,Woburn
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae


## Part 3: Mapping and Clustering Neighbourhoods

### Plot Toronto Map with Neighbourhoods

In [12]:
from geopy.geocoders import Nominatim
address = 'Toronto, Ontario'

##geolocator = Nominatim(user_agent="toronto-neigh") ##Service is sometimes unavailable
##toronto_location = geolocator.geocode(address)
toronto_latitude = 43.6532
toronto_longitude = -79.3832
print('The geograpical coordinates of Toronto are {}, {}.'.format(toronto_latitude, toronto_longitude))

The geograpical coordinates of Toronto are 43.6532, -79.3832.


In [13]:
import folium 

# Create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[toronto_latitude, toronto_longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(df_geocodes['Latitude'], df_geocodes['Longitude'], df_geocodes['Borough'], df_geocodes['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

### Reduce the number of Boroughs to those containing Toronto

In [14]:
borough_toronto = df_geocodes[df_geocodes['Borough'].str.contains('Toronto')]


In [15]:
# create map of borough_toronto using latitude and longitude values
map_Toronto = folium.Map(location=[toronto_latitude, toronto_longitude], zoom_start=12)

for lat, lng, borough, neighborhood in zip(borough_toronto['Latitude'], borough_toronto['Longitude'], borough_toronto['Borough'], borough_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

### Using FourSquare API to explore the Toronto Boroughs

In [16]:
## FourSquare Credentials
CLIENT_ID = 'ID' 
CLIENT_SECRET = 'SECRET' 
VERSION = '20180605' 
LIMIT = 100
radius = 500

In [17]:
## GetNearbyVenues Function to Store Venues in a Neighbourhood
def getNearbyVenues(names, latitudes, longitudes, radius=2500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

toronto_venues = getNearbyVenues(names=borough_toronto['Neighborhood'],
                                   latitudes=borough_toronto['Latitude'],
                                   longitudes=borough_toronto['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The 

In [18]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,The Fox Theatre,43.672801,-79.287272,Indie Movie Theater
1,The Beaches,43.676357,-79.293031,Tori's Bakeshop,43.672114,-79.290331,Vegetarian / Vegan Restaurant
2,The Beaches,43.676357,-79.293031,Ed's Real Scoop,43.672630,-79.287993,Ice Cream Shop
3,The Beaches,43.676357,-79.293031,The Beech Tree,43.680493,-79.288846,Gastropub
4,The Beaches,43.676357,-79.293031,Bagels On Fire,43.672864,-79.286784,Bagel Shop


In [19]:
toronto_venues.shape

(3800, 7)

In [20]:
toronto_venues['Venue Category'].unique()

array(['Indie Movie Theater', 'Vegetarian / Vegan Restaurant',
       'Ice Cream Shop', 'Gastropub', 'Bagel Shop', 'Bakery', 'Park',
       'Coffee Shop', 'Toy / Game Store', 'Health Food Store', 'Beach',
       'Trail', 'Breakfast Spot', 'French Restaurant', 'BBQ Joint', 'Pub',
       'Middle Eastern Restaurant', 'Japanese Restaurant',
       'Hungarian Restaurant', 'Skating Rink', 'Café', 'Liquor Store',
       'Bar', 'Tea Room', 'Caribbean Restaurant', 'Mexican Restaurant',
       'Diner', 'Cupcake Shop', 'Nail Salon', 'Butcher', 'Golf Course',
       'Asian Restaurant', 'Juice Bar', 'Indian Restaurant', 'Gym',
       'Fish & Chips Shop', 'Playground', 'Soccer Field', 'Brewery',
       'Farmers Market', 'Jazz Club', 'Bookstore', 'Gym / Fitness Center',
       'Thai Restaurant', 'Burger Joint', 'Supermarket', 'Sandwich Place',
       'Chocolate Shop', 'Cocktail Bar', 'Italian Restaurant',
       'Ramen Restaurant', 'Pharmacy', 'Filipino Restaurant', 'Bank',
       'Greek Restaurant',

### Neighbourhood Analysis

In [21]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']
pd.DataFrame(toronto_onehot).set_index('Neighborhood')

toronto_onehot.head()

,Afghan Restaurant,Airport,American Restaurant,Amphitheater,Animal Shelter,Aquarium,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Zoo
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Afghan Restaurant,Airport,American Restaurant,Amphitheater,Animal Shelter,Aquarium,Arcade,Art Gallery,Arts & Crafts Store,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Zoo
0,"Adelaide, King, Richmond",0.0,0.00,0.02,0.0,0.0,0.01,0.0,0.02,0.01,...,0.0,0.03,0.0,0.00,0.0,0.00,0.0,0.0,0.02,0.0
1,Berczy Park,0.0,0.00,0.02,0.0,0.0,0.02,0.0,0.00,0.00,...,0.0,0.02,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.0
2,"Brockton, Exhibition Place, Parkdale Village",0.0,0.00,0.02,0.0,0.0,0.00,0.0,0.01,0.01,...,0.0,0.01,0.0,0.02,0.0,0.01,0.0,0.0,0.01,0.0
3,Business Reply Mail Processing Centre 969 Eastern,0.0,0.00,0.03,0.0,0.0,0.00,0.0,0.00,0.01,...,0.0,0.00,0.0,0.02,0.0,0.00,0.0,0.0,0.00,0.0
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.0,0.01,0.00,0.0,0.0,0.02,0.0,0.01,0.00,...,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.02,0.0


### 5 Most Common Venues per Neighbourhood

In [23]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})

----Adelaide, King, Richmond----
----Berczy Park----
----Brockton, Exhibition Place, Parkdale Village----
----Business Reply Mail Processing Centre 969 Eastern----
----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
----Cabbagetown, St. James Town----
----Central Bay Street----
----Chinatown, Grange Park, Kensington Market----
----Christie----
----Church and Wellesley----
----Commerce Court, Victoria Hotel----
----Davisville----
----Davisville North----
----Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West----
----Design Exchange, Toronto Dominion Centre----
----Dovercourt Village, Dufferin----
----First Canadian Place, Underground city----
----Forest Hill North, Forest Hill West----
----Harbord, University of Toronto----
----Harbourfront East, Toronto Islands, Union Station----
----Harbourfront, Regent Park----
----High Park, The Junction South----
----Lawrence Park----
----Little Portugal, Trinity----
----

### Sort in Descending Order

In [24]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [25]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Hotel,Theater,Beer Bar,Pizza Place,Restaurant,Gastropub,Café,Vegetarian / Vegan Restaurant,Gym
1,Berczy Park,Coffee Shop,Café,Hotel,Theater,Italian Restaurant,Farmers Market,Park,Bistro,Restaurant,Japanese Restaurant
2,"Brockton, Exhibition Place, Parkdale Village",Café,Restaurant,Bar,Coffee Shop,Italian Restaurant,Athletics & Sports,Asian Restaurant,Cocktail Bar,Gift Shop,Bakery
3,Business Reply Mail Processing Centre 969 Eastern,Café,Coffee Shop,Brewery,Park,Beach,Bakery,American Restaurant,Italian Restaurant,Indian Restaurant,Bar
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Park,Coffee Shop,Italian Restaurant,Hotel,Café,Bakery,Music Venue,Spa,Beer Bar,Caribbean Restaurant


## Neighbourhood Clustering

In [26]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 0, 3, 1, 1, 1, 0, 0, 1], dtype=int32)

In [27]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [28]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
neighborhoods_venues_sorted.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,"Adelaide, King, Richmond",Coffee Shop,Hotel,Theater,Beer Bar,Pizza Place,Restaurant,Gastropub,Café,Vegetarian / Vegan Restaurant,Gym
1,1,Berczy Park,Coffee Shop,Café,Hotel,Theater,Italian Restaurant,Farmers Market,Park,Bistro,Restaurant,Japanese Restaurant
2,0,"Brockton, Exhibition Place, Parkdale Village",Café,Restaurant,Bar,Coffee Shop,Italian Restaurant,Athletics & Sports,Asian Restaurant,Cocktail Bar,Gift Shop,Bakery
3,3,Business Reply Mail Processing Centre 969 Eastern,Café,Coffee Shop,Brewery,Park,Beach,Bakery,American Restaurant,Italian Restaurant,Indian Restaurant,Bar
4,1,"CN Tower, Bathurst Quay, Island airport, Harbo...",Park,Coffee Shop,Italian Restaurant,Hotel,Café,Bakery,Music Venue,Spa,Beer Bar,Caribbean Restaurant


In [29]:
toronto_merged = borough_toronto.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() 

,Postcode,Latitude,Longitude,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,43.676357,-79.293031,East Toronto,The Beaches,3,Pub,Beach,Coffee Shop,Thai Restaurant,Park,Bakery,Breakfast Spot,BBQ Joint,Bar,Indian Restaurant
41,M4K,43.679557,-79.352188,East Toronto,"The Danforth West, Riverdale",3,Greek Restaurant,Park,Café,Vietnamese Restaurant,Bakery,Ice Cream Shop,Gastropub,Coffee Shop,Italian Restaurant,Pizza Place
42,M4L,43.668999,-79.315572,East Toronto,"The Beaches West, India Bazaar",3,Coffee Shop,Beach,Park,Brewery,Café,Bakery,Gastropub,Ice Cream Shop,Italian Restaurant,American Restaurant
43,M4M,43.659526,-79.340923,East Toronto,Studio District,3,Coffee Shop,Bakery,Park,Café,Brewery,Vietnamese Restaurant,Bar,Thai Restaurant,Italian Restaurant,Restaurant
44,M4N,43.728020,-79.388790,Central Toronto,Lawrence Park,4,Coffee Shop,Italian Restaurant,Bakery,Café,Sporting Goods Shop,Tea Room,Park,Japanese Restaurant,Ice Cream Shop,Grocery Store


In [30]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[toronto_latitude, toronto_longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

#### Cluster 1

In [31]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[4] + list(range(5, toronto_merged.shape[1]))]]


,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
66,"Harbord, University of Toronto",0,Café,Vegetarian / Vegan Restaurant,Sandwich Place,Coffee Shop,Bar,Pizza Place,Yoga Studio,Restaurant,Italian Restaurant,Cocktail Bar
67,"Chinatown, Grange Park, Kensington Market",0,Café,Pizza Place,Bakery,Italian Restaurant,Beer Bar,Yoga Studio,Sandwich Place,Coffee Shop,Bar,Vegetarian / Vegan Restaurant
75,Christie,0,Café,Bar,Coffee Shop,Ice Cream Shop,Vegetarian / Vegan Restaurant,Korean Restaurant,Indian Restaurant,Italian Restaurant,Beer Bar,Pizza Place
76,"Dovercourt Village, Dufferin",0,Café,Coffee Shop,Italian Restaurant,Bar,Park,Bakery,Brewery,Restaurant,Cocktail Bar,Gastropub
77,"Little Portugal, Trinity",0,Café,Bar,Bakery,Cocktail Bar,Pizza Place,Italian Restaurant,Coffee Shop,Yoga Studio,Furniture / Home Store,Asian Restaurant
78,"Brockton, Exhibition Place, Parkdale Village",0,Café,Restaurant,Bar,Coffee Shop,Italian Restaurant,Athletics & Sports,Asian Restaurant,Cocktail Bar,Gift Shop,Bakery
82,"High Park, The Junction South",0,Café,Bar,Bakery,Coffee Shop,Italian Restaurant,Brewery,Park,Pizza Place,Ice Cream Shop,Gastropub


#### Cluster 2

In [33]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[4] + list(range(5, toronto_merged.shape[1]))]]


,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
51,"Cabbagetown, St. James Town",1,Coffee Shop,Park,Diner,Restaurant,Italian Restaurant,Café,Japanese Restaurant,Hotel,Thai Restaurant,Ramen Restaurant
52,Church and Wellesley,1,Coffee Shop,Japanese Restaurant,Park,Restaurant,Italian Restaurant,Pizza Place,Vegetarian / Vegan Restaurant,Gastropub,Theater,Café
53,"Harbourfront, Regent Park",1,Coffee Shop,Park,Café,Japanese Restaurant,Restaurant,Gastropub,Hotel,Farmers Market,Italian Restaurant,Thai Restaurant
54,"Ryerson, Garden District",1,Coffee Shop,Hotel,Café,Japanese Restaurant,Theater,Italian Restaurant,Gastropub,Farmers Market,Pizza Place,Thai Restaurant
55,St. James Town,1,Coffee Shop,Hotel,Café,Theater,Park,Farmers Market,Gastropub,Italian Restaurant,Japanese Restaurant,Thai Restaurant
56,Berczy Park,1,Coffee Shop,Café,Hotel,Theater,Italian Restaurant,Farmers Market,Park,Bistro,Restaurant,Japanese Restaurant
57,Central Bay Street,1,Café,Coffee Shop,Pizza Place,Vegetarian / Vegan Restaurant,Restaurant,Theater,Japanese Restaurant,Concert Hall,Mexican Restaurant,Breakfast Spot
58,"Adelaide, King, Richmond",1,Coffee Shop,Hotel,Theater,Beer Bar,Pizza Place,Restaurant,Gastropub,Café,Vegetarian / Vegan Restaurant,Gym
59,"Harbourfront East, Toronto Islands, Union Station",1,Coffee Shop,Hotel,Café,Park,Beer Bar,Gym,Japanese Restaurant,Baseball Stadium,Steakhouse,Farmers Market
60,"Design Exchange, Toronto Dominion Centre",1,Coffee Shop,Hotel,Theater,Café,Park,Beer Bar,Pizza Place,Farmers Market,Concert Hall,Sporting Goods Shop


#### Cluster 3

In [34]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[4] + list(range(5, toronto_merged.shape[1]))]]


,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
48,"Moore Park, Summerhill East",2,Italian Restaurant,Park,Café,Coffee Shop,Grocery Store,Dessert Shop,American Restaurant,Spa,Indian Restaurant,French Restaurant
49,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",2,Italian Restaurant,Park,Coffee Shop,Café,French Restaurant,Ice Cream Shop,Spa,Middle Eastern Restaurant,American Restaurant,Vegetarian / Vegan Restaurant
50,Rosedale,2,Coffee Shop,Italian Restaurant,Park,Spa,Grocery Store,Restaurant,French Restaurant,Japanese Restaurant,Pizza Place,Hotel
65,"The Annex, North Midtown, Yorkville",2,Coffee Shop,Café,Italian Restaurant,Spa,Park,Grocery Store,Ice Cream Shop,Hotel,Japanese Restaurant,Indian Restaurant


#### Cluster 4

In [35]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[4] + list(range(5, toronto_merged.shape[1]))]]


,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,The Beaches,3,Pub,Beach,Coffee Shop,Thai Restaurant,Park,Bakery,Breakfast Spot,BBQ Joint,Bar,Indian Restaurant
41,"The Danforth West, Riverdale",3,Greek Restaurant,Park,Café,Vietnamese Restaurant,Bakery,Ice Cream Shop,Gastropub,Coffee Shop,Italian Restaurant,Pizza Place
42,"The Beaches West, India Bazaar",3,Coffee Shop,Beach,Park,Brewery,Café,Bakery,Gastropub,Ice Cream Shop,Italian Restaurant,American Restaurant
43,Studio District,3,Coffee Shop,Bakery,Park,Café,Brewery,Vietnamese Restaurant,Bar,Thai Restaurant,Italian Restaurant,Restaurant
83,"Parkdale, Roncesvalles",3,Coffee Shop,Café,Restaurant,Bar,Bakery,Park,Sushi Restaurant,Gastropub,American Restaurant,Pizza Place
87,Business Reply Mail Processing Centre 969 Eastern,3,Café,Coffee Shop,Brewery,Park,Beach,Bakery,American Restaurant,Italian Restaurant,Indian Restaurant,Bar


#### Cluster 5

In [36]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[4] + list(range(5, toronto_merged.shape[1]))]]


,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,Lawrence Park,4,Coffee Shop,Italian Restaurant,Bakery,Café,Sporting Goods Shop,Tea Room,Park,Japanese Restaurant,Ice Cream Shop,Grocery Store
45,Davisville North,4,Coffee Shop,Café,Bakery,Italian Restaurant,Sporting Goods Shop,Park,Sushi Restaurant,Gym,Gastropub,Mexican Restaurant
46,North Toronto West,4,Coffee Shop,Café,Bakery,Italian Restaurant,Park,Sushi Restaurant,Burger Joint,Sporting Goods Shop,Japanese Restaurant,Bookstore
47,Davisville,4,Café,Coffee Shop,Italian Restaurant,Gym,Bakery,Sushi Restaurant,Sporting Goods Shop,Gastropub,Park,Bookstore
63,Roselawn,4,Coffee Shop,Italian Restaurant,Café,Bakery,Japanese Restaurant,Park,Sushi Restaurant,Middle Eastern Restaurant,Pizza Place,Bagel Shop
64,"Forest Hill North, Forest Hill West",4,Café,Coffee Shop,Italian Restaurant,Dessert Shop,Ice Cream Shop,Bakery,Indian Restaurant,Middle Eastern Restaurant,Park,Gastropub
84,"Runnymede, Swansea",4,Italian Restaurant,Coffee Shop,Bar,Bakery,Café,Sushi Restaurant,Park,Gastropub,Pizza Place,Liquor Store
